In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv

load_dotenv()

llm=HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task= "text-generation"
)

model= ChatHuggingFace(llm=llm)

C:\Users\Aritri Baidya\Desktop\MyFiles\Pycharm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#create a state
class LLMState(TypedDict):
    question: str
    answer : str


In [3]:
def llm_qa(state: LLMState) -> LLMState:
    #extract the question from state
    question=state['question']

    #form a prompt
    prompt=f'Answer the following question:\n{question}'

    #ask that question
    answer=model.invoke(prompt).content

    #update the answer in the state
    state['answer']=answer

    return state

In [4]:
#create the graph
graph= StateGraph(LLMState)

#add nodes
graph.add_node('llm_qa',llm_qa)

#add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa',END)

#compile
workflow=graph.compile()

In [6]:
#execute

initial_state={'question': 'How far is moon from earth?'}

final_state=workflow.invoke(initial_state)

print(final_state)


{'question': 'How far is moon from earth?', 'answer': 'The distance between the Earth and the Moon varies due to their elliptical orbits around the Sun. The average distance, known as the mean distance, is approximately 384,400 kilometers (238,855 miles or 238,900 nautical miles. However, this distance can range from a minimum of about 363,104 kilometers (225,622 miles or 121,800 nautical miles (perigee) to a maximum of around 406,747 kilometers (252,865 miles or 154,068 nautical miles (apogee) during its orbit around the Earth. The closest the Moon comes to Earth is during a full moon, which is called a supermoon, and it is called a perigee, and the furthest point in its orbit is called an apogee. During apogee, the distance between the Moon and the Earth can reach up to 406,700,000 kilometers (252,430 miles or 152,780 nautical miles. These variations in distance are known as lunar variations or lunar ranges.'}
